In [ ]:
import numpy as np
import pandas as pd

In [ ]:
prediction=pd.DataFrame(
    columns=["행정동","추정행정동1","결과1","추정행정동2","결과2"]
)
prediction

,행정동,추정행정동1,결과1,추정행정동2,결과2


### 공통

In [ ]:
# p8="/content/drive/MyDrive/asac/project/eda/filtered/v8_노년층연평균소득계산.xlsx"
# v8=pd.read_excel(p8)
# v8.drop(v8.columns[0],axis=1,inplace=True)
# v8

In [ ]:
p15="/content/drive/MyDrive/asac/project/eda/filtered/v15_신한.xlsx"
v15=pd.read_excel(p15)
v15.drop(v15.columns[0],axis=1,inplace=True)
v8=v15

In [ ]:
null_pd=v8[v8["노년층연평균소득"]==0]
null_pd #1
not_null_pd=v8[v8["노년층연평균소득"]!=0]
not_null_pd

In [ ]:
prediction["행정동"]=null_pd.loc[:,"행정동명"]
prediction

,행정동,추정행정동1,결과1,추정행정동2,결과2
0,사직동,NaN,NaN,NaN,NaN
2,부암동,NaN,NaN,NaN,NaN
4,무악동,NaN,NaN,NaN,NaN
5,교남동,NaN,NaN,NaN,NaN
8,종로5.6가동,NaN,NaN,NaN,NaN
...,...,...,...,...,...
418,둔촌1동,NaN,NaN,NaN,NaN
420,암사1동,NaN,NaN,NaN,NaN
421,천호2동,NaN,NaN,NaN,NaN
424,상일1동,NaN,NaN,NaN,NaN


In [ ]:
def collect_dong(gu_name):
  return not_null_pd[not_null_pd["시군구명"]==gu_name]

In [ ]:
target_column="전연령월평균소득"
predict_column="노년층연평균소득"

### 1.총소득이 비슷한 동을 찾아 추정

1. 노년층 연평균 소득이 없는 값들과
2. 동일한 시군구를 뽑아
3. 해당 구에 속하는 열들의 '전 연령평균소득' 중 가장 가까운 값 하나를 골라서
3. 그 값과의 비율을 계산하여 연평균소득 추정


In [ ]:
def get_similar_dong(dongs,target_value,target_column):
  dif=int(1e9)
  similar_name=""
  similar_value=0
  similar_income=0
  for idx, row in dongs.iterrows():
    temp=abs(row[target_column]-target_value)
    if temp<dif:
      dif=temp
      similar_name=row["행정동명"]
      similar_value=row[predict_column]
      similar_income=row[target_column]

  return similar_name,similar_value,similar_income


In [ ]:
v9=v8

In [ ]:
for index,row in null_pd.iterrows():
  gu_name=row["시군구명"]
  dong_name=row["행정동명"]
  target_value=row[target_column]

  dongs_pd=collect_dong(gu_name) #3
  similar_dong_name,similar_dong_value,similar_income=get_similar_dong(dongs_pd,target_value,target_column)

  # 4. 가장 비슷한 동과의 비율통해 소득 추정
  # 전체월평균소득a:전체월평균소득b=노년층연평균소득A:노년층연평균소득추정B
  # B*a=b*A
  # B=(b*A)/a
  predict=(row["전연령월평균소득"]*similar_dong_value)/similar_income

  updated=v8[v8["행정동명"]==dong_name]

  v9.loc[v9["행정동명"]==dong_name,"노년층연평균소득추정"]=predict
  print(v9[v9["행정동명"]==dong_name]["노년층연평균소득추정"])

  prediction.loc[prediction["행정동"]==dong_name,["추정행정동1","결과1"]]=[similar_dong_name,predict]



0    4033.393906
Name: 노년층연평균소득추정, dtype: float64
2    5022.211256
Name: 노년층연평균소득추정, dtype: float64
4    4109.72944
Name: 노년층연평균소득추정, dtype: float64
5    3695.346209
Name: 노년층연평균소득추정, dtype: float64
8    3212.182617
Name: 노년층연평균소득추정, dtype: float64
10    3045.012899
Name: 노년층연평균소득추정, dtype: float64
12    6301.328383
Name: 노년층연평균소득추정, dtype: float64
13    6031.753709
Name: 노년층연평균소득추정, dtype: float64
16    5468.950055
Name: 노년층연평균소득추정, dtype: float64
17    5288.343246
Name: 노년층연평균소득추정, dtype: float64
18    4359.06785
Name: 노년층연평균소득추정, dtype: float64
20    4861.142012
Name: 노년층연평균소득추정, dtype: float64
21    4673.3112
Name: 노년층연평균소득추정, dtype: float64
22    6383.313447
Name: 노년층연평균소득추정, dtype: float64
23    4862.515793
Name: 노년층연평균소득추정, dtype: float64
27    4515.119345
Name: 노년층연평균소득추정, dtype: float64
28    4304.126256
Name: 노년층연평균소득추정, dtype: float64
31    3455.81548
Name: 노년층연평균소득추정, dtype: float64
33    7584.22842
Name: 노년층연평균소득추정, dtype: float64
36    9422.693855
Name: 노년층연평균소득추정, dtype:

In [ ]:
prediction

,행정동,추정행정동1,결과1,추정행정동2,결과2
0,사직동,평창동,4033.393906,NaN,NaN
2,부암동,청운효자동,5022.211256,NaN,NaN
4,무악동,평창동,4109.72944,NaN,NaN
5,교남동,평창동,3695.346209,NaN,NaN
8,종로5.6가동,숭인2동,3212.182617,NaN,NaN
...,...,...,...,...,...
418,둔촌1동,명일1동,4612.128989,NaN,NaN
420,암사1동,길동,7102.864024,NaN,NaN
421,천호2동,성내2동,8427.925791,NaN,NaN
424,상일1동,천호1동,0.0,NaN,NaN


In [ ]:
v9.loc[:,"노년층연평균소득추정"]

0      4033.393906
1              NaN
2      5022.211256
3              NaN
4      4109.729440
          ...     
422            NaN
423            NaN
424            NaN
425            NaN
426            NaN
Name: 노년층연평균소득추정, Length: 427, dtype: float64

In [ ]:
v9.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/v9.xlsx")

### 2.서울시 전역에서 총소득 비슷한 동 찾기


In [ ]:
v9_1=v15

In [ ]:
v9_1

In [ ]:
#서울 전역에서 가장 소득 비슷한 곳 찾기
def get_similar_dong2(target_value,target_column):
  dif=int(1e9)
  similar_name=""
  similar_value=0
  similar_income=0

  for idx, row in not_null_pd.iterrows():
    temp=abs(row[target_column]-target_value)
    if temp<dif:
      dif=temp
      similar_name=row["행정동명"]
      similar_value=row[predict_column]
      similar_income=row[target_column]

  print(similar_name,similar_value)

  return similar_name,similar_value,similar_income


In [ ]:
for index,row in null_pd.iterrows():
  gu_name=row["시군구명"]
  dong_name=row["행정동명"]
  target_value=row[target_column]

  similar_dong_name,similar_dong_value,similar_income=get_similar_dong2(target_value,target_column)

  # 4. 가장 비슷한 동과의 비율통해 소득 추정
  # 전체월평균소득a:전체월평균소득b=노년층연평균소득A:노년층연평균소득추정B
  # B*a=b*A
  # B=(b*A)/a
  predict=(row["전연령월평균소득"]*similar_dong_value)/similar_income

  v9_1.loc[v9_1["행정동명"]==dong_name,"노년층연평균소득추정"]=predict
  print(v9_1[v9_1["행정동명"]==dong_name]["노년층연평균소득추정"])

  prediction.loc[prediction["행정동"]==dong_name,["추정행정동3","결과3"]]=[similar_dong_name,predict]



청담동 4134.418529
0    4114.129636
Name: 노년층연평균소득추정, dtype: float64
하계1동 5197.557873
2    5218.848375
Name: 노년층연평균소득추정, dtype: float64
신사동 6241.738762
4    6235.319175
Name: 노년층연평균소득추정, dtype: float64
고덕1동 8687.357498
5    8663.660905
Name: 노년층연평균소득추정, dtype: float64
개봉3동 4641.052786
8    4647.016967
Name: 노년층연평균소득추정, dtype: float64
독산3동 6311.168328
10    6308.65205
Name: 노년층연평균소득추정, dtype: float64
풍납1동 9889.606606
12    9899.86888
Name: 노년층연평균소득추정, dtype: float64
고척2동 9773.790866
13    9753.018636
Name: 노년층연평균소득추정, dtype: float64
등촌1동 6477.651201
16    6485.544144
Name: 노년층연평균소득추정, dtype: float64
도곡1동 6828.781764
17    6786.724031
Name: 노년층연평균소득추정, dtype: float64
원효로1동 3704.103231
18    3708.740641
Name: 노년층연평균소득추정, dtype: float64
녹번동 5472.051751
20    5473.293985
Name: 노년층연평균소득추정, dtype: float64
성내3동 10368.68327
21    10375.56765
Name: 노년층연평균소득추정, dtype: float64
장지동 3176.197422
22    3167.482866
Name: 노년층연평균소득추정, dtype: float64
녹번동 5472.051751
23    5474.840762
Name: 노년층연평균소득추정, dtype:

In [ ]:
v9_1.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/v16_신한.xlsx")

### 3. 가까운 동을 찾아 그곳과의 비율로 추정

In [ ]:
# !pip install geopy
import geopy.distance

In [ ]:
lpath="/content/drive/MyDrive/asac/project/eda/행정동_위도경도.xlsx"
lpd=pd.read_excel(lpath)
lpd

,행정동명,주소,Latitude,Longitude
0,사직동,서울특별시 종로구 사직동,37.573878,126.970561
1,삼청동,서울특별시 종로구 삼청동,37.590765,126.981016
2,부암동,서울특별시 종로구 부암동,37.594759,126.965589
3,평창동,서울특별시 종로구 평창동,37.613029,126.974503
4,무악동,서울특별시 종로구 무악동,37.576442,126.959311
...,...,...,...,...
421,천호2동,서울특별시 강동구 천호2동,37.544218,127.125015
422,길동,서울특별시 강동구 길동,37.539616,127.145929
423,강일동,서울특별시 강동구 강일동,37.567526,127.172931
424,상일1동,서울특별시 강동구 상일1동,37.550505,127.168367


In [ ]:
geopy.distance.distance((37.573878,126.970561),(37.590765,126.981016)).km

2.0894179513466136

In [ ]:
lpd[lpd["행정동명"]=="사직동"].at[0,"Latitude"]

37.5738782

In [ ]:
# v10=v9
v10=v9_1

In [ ]:
def get_nearest_dong(dongs,target_name,target_column):
  dif=int(1e9)
  similar_name=""
  similar_value=0
  similar_income=0

  latitude=lpd[lpd["행정동명"]==target_name].loc[:,"Latitude"].iat[0]
  longitude=lpd[lpd["행정동명"]==target_name].loc[:,"Longitude"].iat[0]

  for idx, row in dongs.iterrows():
    temp_name=row["행정동명"]

    latitude2=lpd[lpd["행정동명"]==temp_name].loc[:,"Latitude"].iat[0]
    longitude2=lpd[lpd["행정동명"]==temp_name].loc[:,"Longitude"].iat[0]

    temp_dist=geopy.distance.distance((latitude,longitude),(latitude2,longitude2))

    if temp_dist<dif:
      dif=temp_dist
      similar_name=row["행정동명"]
      similar_value=row[predict_column]
      similar_income=row[target_column]

  return similar_name,similar_value,similar_income


In [ ]:
for index,row in null_pd.iterrows():
  gu_name=row["시군구명"]
  dong_name=row["행정동명"]

  dongs_pd=collect_dong(gu_name) #3
  nearest_dong_name,nearest_dong_value,nearest_income=get_nearest_dong(dongs_pd,dong_name,target_column)

  predict2=(row["전연령월평균소득"]*nearest_dong_value)/nearest_income

  v10.loc[v10["행정동명"]==dong_name,"노년층연평균소득추정2"]=predict2

  prediction.loc[prediction["행정동"]==dong_name,["추정행정동2","결과2"]]=[nearest_dong_name,predict2]



In [ ]:
v10.reset_index()
v10
v10.loc[:,"노년층연평균소득추정2"]

0      6911.827148
1         0.000000
2      5022.211256
3         0.000000
4      7042.639566
          ...     
421    3894.147500
422       0.000000
423       0.000000
424       0.000000
425       0.000000
Name: 노년층연평균소득추정2, Length: 426, dtype: float64

In [ ]:
v10.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/v10.xlsx")

In [ ]:
prediction.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/노년층연평균소득추정.xlsx")

### 추정치 값 차이 비교

In [ ]:
prediction["결과차이_1-2"]=abs(prediction["결과1"]-prediction["결과2"])
prediction["결과차이_2-3"]=abs(prediction["결과3"]-prediction["결과2"])

In [ ]:
prediction

,행정동,추정행정동1,결과1,추정행정동2,결과2,추정행정동3,결과3,결과차이_1-2,결과차이_2-3
0,사직동,평창동,4033.393906,청운효자동,6911.827148,청담동,4114.129636,2878.433242,2797.697512
2,부암동,청운효자동,5022.211256,청운효자동,5022.211256,하계1동,5218.848375,0.0,196.637119
4,무악동,평창동,4109.72944,청운효자동,7042.639566,신사동,6235.319175,2932.910127,807.320392
5,교남동,평창동,3695.346209,청운효자동,6332.53157,고덕1동,8663.660905,2637.185361,2331.129335
8,종로5.6가동,숭인2동,3212.182617,이화동,5282.856858,개봉3동,4647.016967,2070.674241,635.839891
...,...,...,...,...,...,...,...,...,...
418,둔촌1동,명일1동,4612.128989,성내3동,12762.861769,문정1동,5760.946359,8150.73278,7001.91541
420,암사1동,길동,7102.864024,암사3동,3783.155943,길동,7102.864024,3319.708082,3319.708082
421,천호2동,성내2동,8427.925791,천호3동,3894.1475,자양1동,9713.019750,4533.778291,5818.87225
424,상일1동,천호1동,0.0,명일2동,0.0,방학2동,0.000000,0.0,0.0


In [ ]:
sorted=prediction.sort_values(by=["결과차이_1-2","결과차이_2-3"],ascending=False)
top_10=sorted.head(10)
top_10

,행정동,추정행정동1,결과1,추정행정동2,결과2,추정행정동3,결과3,결과차이_1-2,결과차이_2-3
286,여의동,당산2동,5277.383222,신길1동,14096.526054,목1동,5976.439752,8819.142832,8120.086302
418,둔촌1동,명일1동,4612.128989,성내3동,12762.861769,문정1동,5760.946359,8150.73278,7001.91541
378,개포2동,신사동,6251.124416,개포1동,14116.7773,신사동,6251.124416,7865.652884,7865.652884
411,암사2동,명일1동,4766.176124,천호1동,12176.088921,청림동,5050.996423,7409.912797,7125.092498
301,영등포동,신길7동,3450.424468,신길3동,10215.865284,돈암1동,4369.163159,6765.440816,5846.702126
219,서강동,상암동,4305.73805,서교동,10489.081714,송파2동,8986.416043,6183.343664,1502.665671
230,신월5동,신정4동,12132.391836,신월1동,5950.11126,신정4동,12132.391836,6182.280576,6182.280576
92,이문2동,휘경2동,5523.627877,회기동,11640.930275,화곡3동,5817.262771,6117.302398,5823.667504
402,잠실7동,잠실4동,5904.705125,잠실본동,11777.71123,목5동,6525.592222,5873.006105,5252.119008
10,창신1동,숭인2동,3045.012899,창신2동,8722.707309,독산3동,6308.652050,5677.69441,2414.055259


In [ ]:
prediction.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/노년층연평균소득추정.xlsx")